# **Лабораторна робота 6: Пошук аномалій та вирішення задачі *anomaly detection* за допомогою бібліотек `scikit-learn`та `PyTorch`**
**Всі завдання виконуються індивідуально. Використання запозиченого коду буде оцінюватись в 0 балів.**

**Лабораторні роботи де в коді буде використаня КИРИЛИЦІ будуть оцінюватись в 20 балів.**

### Мета роботи:
Ознайомитися з основними методами виявлення аномалій, навчитися використовувати бібліотеки `scikit-learn` та `PyTorch` для реалізації алгоритмів пошуку аномалій, проаналізувати ефективність різних методів на реальних наборах даних з Kaggle.


### Опис завдання:

1. **Постановка задачі**:
   Використовуючи один із доступних наборів даних Kaggle (наприклад, *Credit Card Fraud Detection*, *Network Intrusion*, або інші), вам потрібно розв'язати задачу виявлення аномалій. Основна мета — ідентифікувати аномальні записи серед нормальних. Вибраний набір даних повинен містити мітки аномалій для перевірки результатів.

2. **Етапи виконання завдання**:
   - Завантажте та підготуйте набір даних.
   - Проведіть попередню обробку даних (масштабування, заповнення пропущених значень, видалення нерелевантних ознак).
   - Використайте різні методи виявлення аномалій:
     - **Методи з бібліотеки scikit-learn**:
       - Isolation Forest
       - One-Class SVM
       - Local Outlier Factor (LOF)
     - **Методи з використанням PyTorch**:
       - Автоенкодери для виявлення аномалій.
   - Порівняйте отримані результати, обчисліть метрики якості (Precision, Recall, F1-Score).
   - Оцініть, який метод найкраще підходить для вирішення задачі на вашому наборі даних.

### Покрокова інструкція

1. **Підготовка середовища**:
   - Встановіть необхідні бібліотеки:
     ```
     pip install scikit-learn torch pandas numpy matplotlib
     ```

2. **Вибір набору даних з Kaggle**:
   Зареєструйтесь на Kaggle та оберіть один із наборів даних для виявлення аномалій. Наприклад:
   - [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud)
   - [Network Intrusion Detection](https://www.kaggle.com/xyuanh/benchmarking-datasets)

3. **Попередня обробка даних**:
   - Завантажте дані та проведіть їхню початкову обробку.
   - Масштабуйте ознаки за допомогою `StandardScaler` або `MinMaxScaler`.
   - Розділіть дані на навчальну і тестову вибірки.

4. **Методи з бібліотеки `scikit-learn`**:

   - **Isolation Forest**:
     ```
     from sklearn.ensemble import IsolationForest
     ```

   - **One-Class SVM**:
     ```
     from sklearn.svm import OneClassSVM
     ```

   - **Local Outlier Factor**:
     ```
     from sklearn.neighbors import LocalOutlierFactor
     ```

5. **Методи на основі нейронних мереж (PyTorch)**:

   Використайте автоенкодер для пошуку аномалій. Побудуйте нейронну мережу з енкодером і декодером. Під час навчання порівняйте відновлені дані з вхідними та обчисліть помилку. Записи з великою помилкою можуть бути аномаліями.

   - **Реалізація автоенкодера**:
     ```
     import torch
     import torch.nn as nn
     import torch.optim as optim
     ```

6. **Оцінка результатів**:
   Використовуйте метрики оцінки якості:
   - `Precision`, `Recall`, `F1-score`
   ```
   from sklearn.metrics import classification_report
   ```

7. **Звіт**:
   - Поясніть, який метод дав найкращі результати.
   - Проаналізуйте, чому деякі методи працюють краще на вашому наборі даних.
   - Оцініть можливості використання глибоких нейронних мереж (автоенкодерів) для вирішення задачі.


### Результати, які необхідно надати:
1. Код рішення у вигляді Jupyter Notebook з аналізом результатів та поясненнями.


### Дедлайн:
[23 жовтня 23:59]


### Корисні ресурси:
- [Документація PyTorch](https://pytorch.org/docs/stable/index.html)
- [Документація scikit-learn](https://scikit-learn.org/stable/documentation.html)
- [Kaggle Datasets](https://www.kaggle.com/datasets)

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('creditcard.csv')

print(data.head())
print(data.info())

scaler = StandardScaler()
data[['scaled_amount', 'scaled_time']] = scaler.fit_transform(data[['Amount', 'Time']])
data = data.drop(['Amount', 'Time'], axis=1)

X = data.drop('Class', axis=1)
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [4]:
isolation_forest = IsolationForest(contamination=0.001, random_state=42)
y_pred_train = isolation_forest.fit_predict(X_train)
y_pred_test = isolation_forest.predict(X_test)

y_pred_test = [1 if x == -1 else 0 for x in y_pred_test]

print("Isolation Forest:")
print(classification_report(y_test, y_pred_test))

Isolation Forest:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.35      0.24      0.29        98

    accuracy                           1.00     56962
   macro avg       0.68      0.62      0.64     56962
weighted avg       1.00      1.00      1.00     56962



In [13]:
oc_svm = OneClassSVM(gamma='auto', nu=0.001)
oc_svm.fit(X_train)

y_pred_svm = oc_svm.predict(X_test)
y_pred_svm = [1 if x == -1 else 0 for x in y_pred_svm]

print("One-Class SVM:")
print(classification_report(y_test, y_pred_svm))

One-Class SVM:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     56864
           1       0.07      0.41      0.12        98

    accuracy                           0.99     56962
   macro avg       0.53      0.70      0.56     56962
weighted avg       1.00      0.99      0.99     56962



In [14]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.001)
y_pred_lof = lof.fit_predict(X_test)
y_pred_lof = [1 if x == -1 else 0 for x in y_pred_lof]

print("Local Outlier Factor:")
print(classification_report(y_test, y_pred_lof))

Local Outlier Factor:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.04      0.02      0.03        98

    accuracy                           1.00     56962
   macro avg       0.52      0.51      0.51     56962
weighted avg       1.00      1.00      1.00     56962



In [7]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(X_train.shape[1], 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8)
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, X_train.shape[1])
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
X_train_tensor = torch.FloatTensor(X_train.values)
X_test_tensor = torch.FloatTensor(X_test.values)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train_tensor)
    loss = criterion(outputs, X_train_tensor)
    
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')

Epoch 1, Loss: 1.1014
Epoch 2, Loss: 1.1003
Epoch 3, Loss: 1.0993
Epoch 4, Loss: 1.0983
Epoch 5, Loss: 1.0974
Epoch 6, Loss: 1.0965
Epoch 7, Loss: 1.0957
Epoch 8, Loss: 1.0948
Epoch 9, Loss: 1.0940
Epoch 10, Loss: 1.0932
Epoch 11, Loss: 1.0924
Epoch 12, Loss: 1.0916
Epoch 13, Loss: 1.0908
Epoch 14, Loss: 1.0900
Epoch 15, Loss: 1.0892
Epoch 16, Loss: 1.0883
Epoch 17, Loss: 1.0874
Epoch 18, Loss: 1.0864
Epoch 19, Loss: 1.0854
Epoch 20, Loss: 1.0844


In [16]:
model.eval()
with torch.no_grad():
    reconstructions = model(X_test_tensor)
    loss = torch.mean((reconstructions - X_test_tensor) ** 2, dim=1)

threshold = torch.quantile(loss, 0.99).item()
y_pred_autoencoder = (loss > threshold).int().numpy()

print("Autoencoder:")
print(classification_report(y_test, y_pred_autoencoder))

Autoencoder:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56864
           1       0.10      0.56      0.16        98

    accuracy                           0.99     56962
   macro avg       0.55      0.78      0.58     56962
weighted avg       1.00      0.99      0.99     56962



In [18]:
precision_scores = []
recall_scores = []
f1_scores = []

methods = ['Isolation Forest', 'One-Class SVM', 'LOF', 'Autoencoder']
predictions = [y_pred_test, y_pred_svm, y_pred_lof, y_pred_autoencoder]

for y_pred in predictions:
    precision_scores.append(precision_score(y_test, y_pred, zero_division=0))
    recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
    f1_scores.append(f1_score(y_test, y_pred, zero_division=0))


print("Metrics for each method:")
for i, method in enumerate(methods):
    print(f"{method}: Precision={precision_scores[i]:.4f}, Recall={recall_scores[i]:.4f}, F1-Score={f1_scores[i]:.4f}")


Metrics for each method:
Isolation Forest: Precision=0.0965, Recall=0.5612, F1-Score=0.1647
One-Class SVM: Precision=0.0677, Recall=0.4082, F1-Score=0.1161
LOF: Precision=0.0351, Recall=0.0204, F1-Score=0.0258
Autoencoder: Precision=0.0965, Recall=0.5612, F1-Score=0.1647


**Висновок:**

1. Найкращий метод: Isolation Forest показав найкращі результати з Precision 0.35, Recall 0.24 та F1-Score 0.29. Він досяг балансу між точністю та здатністю виявляти аномалії, хоча результати все ще далекі від ідеальних.

2. Причина кращої роботи деяких методів:
   - Isolation Forest гарно виявляє аномалії в даних і не вимагає нормального розподілу даних.
   - One-Class SVM має високий Recall (0.41), але низький Precision (0.07), що вказує на багато хибних спрацювань.
   - Local Outlier Factor (LOF) не так добре працює на великих наборах даних.
   
3. Оцінка використання автоенкодерів: Автоенкодер досяг Recall 0.56 (краще, ніж інші методи), але має низький 
Precision 0.10, що свідчить про багато хибних спрацювань.

Загальний висновок: Isolation Forest є найбільш збалансованим методом для цього завдання, але якщо важливо не пропускати аномалії, автоенкодер також може бути корисним.